# Extract Data from [Football API](https://www.api-football.com/documentation-v3#tag/Players/operation/get-players)

*See player_stats.py for details on functions used below*

## 1. Pull and save all necessary data from API

In [ ]:
import player_stats

# Leagues Ids
leagues = [310,
 129,
 131,
 134,
 188,
 218,
 418,
 116,
 144,
 315,
 71,
 72,
 75,
 76,
 172,
 265,
 169,
 239,
 240,
 162,
 210,
 318,
 345,
 119,
 242,
 39,
 40,
 41,
 42,
 45,
 46,
 47,
 48,
 244,
 61,
 62,
 66,
 327,
 78,
 79,
 81,
 197,
 234,
 271,
 164,
 290,
 357,
 383,
 135,
 136,
 137,
 98,
 389,
 371,
 262,
 88,
 407,
 103,
 304,
 250,
 252,
 281,
 106,
 94,
 95,
 305,
 283,
 235,
 179,
 332,
 373,
 292,
 140,
 141,
 143,
 113,
 207,
 203,
 301,
 268,
 270,
 253,
 299,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 9,
 11,
 12,
 13,
 15]

# convert league ids to string
leagues = [str(league) for league in leagues]

print(type(leagues[0]))
print(len(leagues))

The below code will save json files to the current directory called "seasons.json" and "players.json".

Only call this funciton to get/update the data in the saved files. This allows us to have access to the data locally without making multiple API calls each time we need the data. DO NOT need to run this code if you already have the "players.json" file.

WANING: May take up to 10hrs to run

In [ ]:
###########################################
#  RUN THIS ONCE TO UPDATE THE JSON FILES #
###########################################

player_stats.update_files(leagues)

###########################################
#  RUN THIS ONCE TO UPDATE THE JSON FILES #
###########################################

## 2. Load data from json files

In [ ]:
import json

# load seasons
with open("seasons.json", "r") as file:
    seasons = file.readlines()
    seasons = [year.strip("\n") for year in seasons]
    
# load teams
# with open("teams.json", "r") as file:
#     teams = json.loads(file.read())

# load players
with open("players.json", "r") as file:
    players = json.loads(file.read())

print("Num leagues:", len(leagues))
print("Num seasons:", len(seasons))
# print("Num teams per league and season:", len(teams))
print("Num players per league season:", len(players))

In [ ]:
# look at data format
print(players[0].keys())
print(players[0]['player'].keys())
print(players[0]['statistics'][0].keys())
print()

# example player
print(players[30]['player']['name'])
# one set of statistics per team player belonged to in the season
print(players[30]['statistics'][0])
print(players[30]['statistics'][1])

## 3. Convert data to pandas table and export to CSV

In [ ]:
import pandas as pd

# flatten the list of players (list of nested dicts) into pandas table
df1 = pd.json_normalize(players, sep="_")
# duplicate rows for each statistic a player has
df1 = df1.explode("statistics").reset_index(drop=True)

# df1 rows are each combination of player, league, team
# df1 cols are all the player info and statistcs (note the statistics column needs to be reformatted)
df1.shape

In [ ]:
# flatten the list of statistics of player into another pandas table
df2 = pd.json_normalize(df1["statistics"], sep="_").reset_index(drop=True)

# df1 rows are each combination of player, league, team
# df1 cols are all the player statistics
df2.shape

In [ ]:
# concatenate the player info and player statistics tables into one
final_df = pd.concat([df1.drop(["statistics"], axis=1), df2], axis=1)
print(final_df.shape)
final_df.head()

In [ ]:
# save dataframe as csv
final_df.to_csv("player_stats.csv", index=False, encoding="utf-8-sig")

## 4. Data Pre-Processing

TO-DO: Need to consider which rows to drop (and for which columns / types of players) since there are many NaN

In [19]:
import pandas as pd
players_df_og = pd.read_csv('player_stats.csv')

C:\Users\Yash\AppData\Local\Temp\ipykernel_35736\1892093246.py:2: DtypeWarning: Columns (20,21,28) have mixed types. Specify dtype option on import or set low_memory=False.
  players_df_og = pd.read_csv('player_stats.csv')


In [20]:
pd.set_option('display.max_columns', None)
players_df_og.head(10)

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,player_weight,player_injured,player_photo,team_id,team_name,team_logo,league_id,league_name,league_country,league_logo,league_flag,league_season,games_appearences,games_lineups,games_minutes,games_number,games_position,games_rating,games_captain,substitutes_in,substitutes_out,substitutes_bench,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
0,4299,Endri Bakiu,Endri,Bakiu,32.0,1987-01-06,Tiranë,Albania,Albania,NaN,NaN,False,https://media-2.api-sports.io/football/players...,3325.0,Bylis,https://media-1.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Attacker,NaN,False,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,142562,Pedro Paulo Pinheiro Neves,Pedro Paulo,Pinheiro Neves,39.0,1981-10-06,Lisboa,Portugal,Portugal,178 cm,66 kg,False,https://media-1.api-sports.io/football/players...,3325.0,Bylis,https://media-2.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Midfielder,NaN,False,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,210892,Roland Dervishi,Roland,Dervishi,34.0,1982-02-16,Peqin,Albania,Albania,NaN,NaN,False,https://media-1.api-sports.io/football/players...,3835.0,Shkumbini Peqin,https://media-1.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Attacker,NaN,False,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,210924,Gersi Arbri,Gersi,Arbri,33.0,1984-08-01,Berat,Albania,Albania,NaN,NaN,False,https://media-1.api-sports.io/football/players...,3332.0,Lushnja,https://media-2.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Midfielder,NaN,False,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,211075,Klodian Arbri,Klodian,Arbri,37.0,1979-09-10,Berat,Albania,Albania,NaN,NaN,False,https://media-1.api-sports.io/football/players...,3332.0,Lushnja,https://media-1.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Attacker,NaN,False,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,211113,Endri Dalipi,Endri,Dalipi,32.0,1983-05-02,Elbasan,Albania,Albania,NaN,NaN,False,https://media-2.api-sports.io/football/players...,3325.0,Bylis,https://media-2.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Midfielder,NaN,False,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,211139,Washington Luis Pinto Galvão,Washington Luis,Pinto Galvão,31.0,1983-07-05,NaN,Brazil,Brazil,NaN,NaN,False,https://media-1.api-sports.io/football/players...,3321.0,Apolonia Fier,https://media-1.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Attacker,NaN,False,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,211187,Eriol Merxha,Eriol,Merxha,36.0,1979-04-21,Elbasan,Albania,Albania,NaN,NaN,False,https://media-2.api-sports.io/football/players...,3328.0,Elbasani,https://media-1.api-sports.io/football/teams/3...,NaN,Superliga,NaN,NaN,NaN,2008-2009,NaN,NaN,NaN,NaN,Midfielder,NaN,False,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [21]:
# # Display the first 5 rows of the DataFrame
# print('--- First 5 rows ---')
# print(players_df.head())

# # Display the last 5 rows of the DataFrame
# print('--- Last 5 rows ---')
# print(players_df.tail())

# Display the shape of the DataFrame (number of rows and columns)
print('--- Shape of the DataFrame ---')
print(players_df_og.shape)

# Display basic summary statistics of the DataFrame
print('--- Summary statistics ---')
print(players_df_og.describe())

# Display information about the data types and missing values in the DataFrame
print('--- Information about the DataFrame ---')
print(players_df_og.info())

--- Shape of the DataFrame ---
(762670, 59)
--- Summary statistics ---
           player_id     player_age        team_id      league_id  \
count  762670.000000  756570.000000  759327.000000  582930.000000   
mean    81458.783289      30.751151    1707.909798     120.372791   
std     83946.831932       5.466834    2483.798193      99.771486   
min         1.000000       0.000000       1.000000       1.000000   
25%     19637.000000      27.000000     339.000000      45.000000   
50%     50748.000000      31.000000     794.000000      81.000000   
75%    111590.000000      35.000000    2246.000000     179.000000   
max    412650.000000      67.000000   21187.000000     418.000000   

       games_appearences  games_lineups  games_minutes  games_number  \
count      751120.000000  746363.000000  745606.000000           0.0   
mean            9.564910       7.501154     673.637668           NaN   
std            10.943439       9.850654     864.929183           NaN   
min             0.0

In [22]:
players_df_og.isna().sum()

player_id                     0
player_name                  19
player_firstname             41
player_lastname              41
player_age                 6100
player_birth_date          6100
player_birth_place       135459
player_birth_country         41
player_nationality           41
player_height            126065
player_weight            172035
player_injured                0
player_photo                  0
team_id                    3343
team_name                  3343
team_logo                  3343
league_id                179740
league_name                   2
league_country           179740
league_logo              179740
league_flag              262983
league_season                 2
games_appearences         11550
games_lineups             16307
games_minutes             17064
games_number             762670
games_position              151
games_rating             594833
games_captain                 2
substitutes_in            16307
substitutes_out           16364
substitu

In [23]:
#Step 1 - Remove rows with Nans and 0 in minutes played
players_df = players_df_og.loc[~(players_df_og['games_minutes'].isna() | (players_df_og['games_minutes'] == 0))]

In [24]:
#Step 2 - Remove invalid data in player positions
# Define a list of valid position values
valid_positions = ['Attacker', 'Midfielder', 'Defender', 'Goalkeeper']

# Filter rows with invalid position values
players_df = players_df.loc[players_df['games_position'].isin(valid_positions)]

In [25]:
#Step 3 - Remove data points that do not fit real world intuition
# Remove rows with "Attacker", "Midfielder", or "Defender" and less than 50 in "passes_total" column
players_df = players_df.loc[~((players_df['games_position'].isin(['Attacker', 'Midfielder', 'Defender'])) & (players_df['passes_total'] < 50))]

# Remove rows with "Goalkeeper" and less than 1 in "goals_saves" column
players_df = players_df.loc[~((players_df['games_position'] == 'Goalkeeper') & (players_df['goals_saves'] < 1))]

In [26]:
#Step 4 - Remove rows with Nans in vital attribute columns
# Remove rows with "Attacker", "Midfielder", or "Defender" and NaN in "passes_total" column
players_df = players_df.loc[~((players_df['games_position'].isin(['Attacker', 'Midfielder', 'Defender'])) & (players_df['passes_total'].isna()))]

# Remove rows with "Goalkeeper" and NaN in "goals_saves" column
players_df = players_df.loc[~((players_df['games_position'] == 'Goalkeeper') & (players_df['goals_saves'].isna()))]

In [27]:
#Step 5 - Remove rows with negative values in game minutes column
players_df = players_df[players_df['games_minutes'] >= 0]

In [28]:
# Create game stats
players_df['passes_90mins'] = players_df['passes_total']/players_df['games_minutes']*90
players_df['saves_90mins'] = players_df['goals_saves']/players_df['games_minutes']*90

In [29]:
players_df['count_(player)'] = players_df.groupby('player_id')['player_id'].transform('count')

players_df['pass_threshold'] = ((players_df['passes_90mins'] > 160) | (players_df['passes_90mins'] < 5)) & (players_df['games_position'] != 'Goalkeeper')
players_df['pass_threshold'] = players_df[players_df['pass_threshold']].groupby('player_id')['pass_threshold'].transform('sum')
players_df['pass_threshold'].fillna(0, inplace=True)
players_df['pass_threshold'] = players_df['pass_threshold'].astype(int)

players_df['save_threshold'] = ((players_df['saves_90mins'] > 10) | (players_df['saves_90mins'] < 1)) & (players_df['games_position'] == 'Goalkeeper')
players_df['save_threshold'] = players_df[players_df['save_threshold']].groupby('player_id')['save_threshold'].transform('sum')
players_df['save_threshold'].fillna(0, inplace=True)
players_df['save_threshold'] = players_df['save_threshold'].astype(int)

In [31]:
columns_to_fill = ['substitutes_in', 'substitutes_out', 'substitutes_bench', 'shots_total', 'shots_on', 'goals_total', 'goals_conceded', 'goals_assists', 'goals_saves', 'passes_total', 'passes_key', 'passes_accuracy', 'tackles_total', 'tackles_blocks', 'tackles_interceptions', 'duels_total', 'duels_won', 'dribbles_attempts', 'dribbles_success', 'dribbles_past', 'fouls_drawn', 'fouls_committed', 'cards_yellow', 'cards_yellowred', 'cards_red', 'penalty_won', 'penalty_commited', 'penalty_scored', 'penalty_missed', 'penalty_saved']

players_df[columns_to_fill] = players_df[columns_to_fill].fillna(0)

In [32]:
#Temporary step (only 100 rows deleted as a result)
players_df = players_df[((players_df['games_position'] == 'Goalkeeper') 
                         & (players_df['count_(player)'] != players_df['save_threshold'])) |
                         ((players_df['games_position'] != 'Goalkeeper') 
                          & (players_df['count_(player)'] != players_df['pass_threshold']))]

In [33]:
# create the new column
players_df['passes_90min_I'] = players_df['passes_90mins']

# define the condition
condition = (players_df['games_position'] != 'Goalkeeper') & (players_df['pass_threshold'] < players_df['count_(player)'])

# loop through each player_id and update the passes_90min_I column
for player_id in players_df['player_id'].unique():
    mask = players_df['player_id'] == player_id
    player_mask = condition & mask
    if any(player_mask):
        player_avg = players_df.loc[mask & (players_df['passes_90mins'].between(5, 160)), 'passes_90mins'].mean()
        players_df.loc[player_mask, 'passes_90min_I'] = players_df.loc[player_mask, 'passes_90mins'].apply(
            lambda x: player_avg if x > 160 or x < 5 else x)

In [34]:
# create the new column
players_df['saves_90min_I'] = players_df['saves_90mins']

# define the condition
condition = (players_df['games_position'] == 'Goalkeeper') & (players_df['save_threshold'] < players_df['count_(player)'])

# loop through each player_id and update the passes_90min_I column
for player_id in players_df['player_id'].unique():
    mask = players_df['player_id'] == player_id
    player_mask = condition & mask
    if any(player_mask):
        player_avg = players_df.loc[mask & (players_df['saves_90mins'].between(1, 10)), 'saves_90mins'].mean()
        players_df.loc[player_mask, 'saves_90min_I'] = players_df.loc[player_mask, 'saves_90mins'].apply(
            lambda x: player_avg if x > 10 or x < 1 else x)

In [35]:
# feature list for clustering 
feature_list = ['games_appearences', 'substitutes_in', 'substitutes_out',
       'substitutes_bench', 'passes_90min_I', 'saves_90min_I',
       'shots_90mins', 'shots_on_90mins', 'goals_total_90mins',
       'goals_conceded_90mins', 'goals_assists_90mins', 'goals_saves_90mins',
       'passes_total_90mins', 'passes_key_90mins', 'passes_accuracy_90mins',
       'tackles_total_90mins', 'tackles_blocks_90mins',
       'tackles_interceptions_90mins', 'duels_total_90mins',
       'duels_won_90mins', 'dribbles_attempts_90mins',
       'dribbles_success_90mins', 'dribbles_past_90mins', 'fouls_drawn_90mins',
       'fouls_committed_90mins', 'cards_yellow_90mins',
       'cards_yellowred_90mins', 'cards_red_90mins', 'penalty_won_90mins',
       'penalty_commited_90mins', 'penalty_scored_90mins',
       'penalty_missed_90mins', 'penalty_saved_90mins']

In [36]:
players_df

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,player_weight,player_injured,player_photo,team_id,team_name,team_logo,league_id,league_name,league_country,league_logo,league_flag,league_season,games_appearences,games_lineups,games_minutes,games_number,games_position,games_rating,games_captain,substitutes_in,substitutes_out,substitutes_bench,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,passes_90mins,saves_90mins,count_(player),pass_threshold,save_threshold,shots_90mins,shots_on_90mins,goals_total_90mins,goals_conceded_90mins,goals_assists_90mins,goals_saves_90mins,passes_total_90mins,passes_key_90mins,passes_accuracy_90mins,tackles_total_90mins,tackles_blocks_90mins,tackles_interceptions_90mins,duels_total_90mins,duels_won_90mins,dribbles_attempts_90mins,dribbles_success_90mins,dribbles_past_90mins,fouls_drawn_90mins,fouls_committed_90mins,cards_yellow_90mins,cards_yellowred_90mins,cards_red_90mins,penalty_won_90mins,penalty_commited_90mins,penalty_scored_90mins,penalty_missed_90mins,penalty_saved_90mins,passes_90min_I,saves_90min_I
24746,6945,G. Moss,Glen,Moss,37.0,1983-01-19,Hastings,New Zealand,New Zealand,188 cm,82 kg,False,https://media-1.api-sports.io/football/players...,942.0,Wellington Phoenix,https://media-1.api-sports.io/football/teams/9...,188.0,A-League,Australia,https://media-2.api-sports.io/football/leagues...,https://media-1.api-sports.io/flags/au.svg,2014,27.0,27.0,2430.0,NaN,Goalkeeper,6.900000,False,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,6.0,4.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.148148,0.222222,6,0,1,0.000000,0.000000,0.000000,0.111111,NaN,0.222222,0.148148,0.000000,1.037037,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.00000,NaN,NaN,0.0,0.0,0.0,0.148148,3.083884
24766,2753,M. Milligan,Mark,Milligan,36.0,1985-08-04,Sydney,Australia,Australia,178 cm,78 kg,False,https://media-1.api-sports.io/football/players...,944.0,Melbourne Victory,https://media-2.api-sports.io/football/teams/9...,188.0,A-League,Australia,https://media-2.api-sports.io/football/leagues...,https://media-2.api-sports.io/flags/au.svg,2014,20.0,20.0,1782.0,NaN,Midfielder,6.733333,False,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,75.0,1.0,54.0,0.0,0.0,5.0,22.0,12.0,3.0,2.0,0.0,4.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.787879,NaN,6,1,0,0.000000,0.000000,0.050505,NaN,NaN,NaN,3.787879,0.050505,2.727273,NaN,NaN,0.252525,1.111111,0.606061,0.151515,0.101010,NaN,0.202020,0.202020,0.303030,0.000000,0.00000,NaN,NaN,0.0,0.0,NaN,53.801384,NaN
24769,6863,C. Valeri,Carl,Valeri,37.0,1984-08-14,Canberra,Australia,Australia,183 cm,75 kg,False,https://media-2.api-sports.io/football/players...,944.0,Melbourne Victory,https://media-1.api-sports.io/football/teams/9...,188.0,A-League,Australia,https://media-1.api-sports.io/football/leagues...,https://media-2.api-sports.io/flags/au.svg,2014,29.0,29.0,2549.0,NaN,Midfielder,6.533333,False,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,59.0,2.0,54.0,0.0,0.0,1.0,15.0,7.0,0.0,0.0,0.0,3.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.083170,NaN,5,1,0,0.000000,0.000000,0.070616,NaN,NaN,NaN,2.083170,0.070616,1.906630,NaN,NaN,0.035308,0.529619,0.247156,0.000000,0.000000,NaN,0.105924,0.070616,0.141232,0.035308,0.00000,NaN,NaN,0.0,0.0,NaN,37.307592,NaN
24770,6946,N. Boogaard,Nigel,Boogaard,35.0,1986-08-14,Sydney,Australia,Australia,188 cm,87 kg,False,https://media-1.api-sports.io/football/players...,948.0,Adelaide United,https://media-1.api-sports.io/football/teams/9...,188.0,A-League,Australia,https://media-1.api-sports.io/football/le

In [37]:
players_df.to_csv('../analytics/players_preprocessed1.csv', index=False)